# Test om koden er riktig implementert

Her er et forslag til testfunksjoner for å sjekke om koden er riktig implementert.
```assert variabel``` vil gi en feilmelding med mindre variabelen ```variabel = True```. For eksempel vil ```assert a == b``` gi en feilmelding med mindre ```a``` og ```b``` er like.

In [5]:
#For eksempel:
variable = True
assert variable, "You need to change 'variable' to True"

In [6]:
from layers import *
from neural_network import NeuralNetwork
from utils import onehot
import numpy as np

In [7]:
#We choose some arbitrary values for the dimensions
b = 6
n_max = 7
m = 8
n = 5

d = 10
k = 3
p = 20

#Create an arbitrary dataset
x = np.random.randint(0, m, (b,n))
y = np.random.randint(0, m, (b,n_max))

print(x)
print(y)

#initialize the layers
feed_forward = FeedForward(d,p)
attention = Attention(d,k) # Changed from k to n to n_max
embed_pos = EmbedPosition(n_max,m,d)
un_embed = LinearLayer(d,m)
softmax = Softmax()


#a manual forward pass
X = onehot(x, m)
z0 = embed_pos.forward(X)
z1 = feed_forward.forward(z0)
z2 = attention.forward(z1)
z3 = un_embed.forward(z2)
Z = softmax.forward(z3) 

z = np.argmax(Z,axis=1)
print(f"big z: {Z[0]}")
print(f"z: {z[0]}")

#check the shapes
assert X.shape == (b,m,n), f"X.shape={X.shape}, expected {(b,m,n)}"
assert z0.shape == (b,d,n), f"z0.shape={z0.shape}, expected {(b,d,n)}"
assert z1.shape == (b,d,n), f"z1.shape={z1.shape}, expected {(b,d,n)}"
assert z2.shape == (b,d,n), f"z2.shape={z2.shape}, expected {(b,d,n)}"
assert z3.shape == (b,m,n), f"z3.shape={z3.shape}, expected {(b,m,n)}"
assert Z.shape == (b,m,n), f"Z.shape={Z.shape}, expected {(b,m,n)}"

#is X one-hot?
assert X.sum() == b*n, f"X.sum()={X.sum()}, expected {b*n}"


assert np.allclose(Z.sum(axis=1), 1), f"Z.sum(axis=1)={Z.sum(axis=1)}, expected {np.ones(b)}"
assert np.abs(Z.sum() - b*n) < 1e-5, f"Z.sum()={Z.sum()}, expected {b*n}"
assert np.all(Z>=0), f"Z={Z}, expected all entries to be non-negative"


[[4 6 4 2 2]
 [7 1 1 3 5]
 [5 7 1 5 3]
 [5 1 6 6 1]
 [0 2 1 6 4]
 [2 1 0 7 1]]
[[5 3 1 6 6 5 0]
 [7 4 1 6 7 5 7]
 [1 5 3 3 2 4 3]
 [1 1 6 2 3 1 5]
 [2 7 1 4 5 6 0]
 [5 7 6 0 2 1 0]]
softmax input shape: (6, 5, 5)
softmax input shape: (6, 8, 5)
big z: [[0.12871725 0.11818143 0.13439672 0.11357661 0.12401323]
 [0.12207864 0.12787055 0.12252554 0.13774126 0.13000427]
 [0.13271785 0.128682   0.12572756 0.12226124 0.11785556]
 [0.12737425 0.12250566 0.12007294 0.13485759 0.1316127 ]
 [0.12885488 0.12646306 0.13080904 0.13866544 0.14101334]
 [0.11976129 0.13128253 0.12059868 0.11089505 0.11302105]
 [0.11405198 0.12728858 0.11854543 0.13203403 0.12732658]
 [0.12644385 0.11772619 0.12732408 0.10996879 0.11515327]]
z: [2 5 0 4 4]


In [8]:

#test the forward pass
x = np.random.randint(0, m, (b,n_max))
X = onehot(x, m)

#we test with a y that is shorter than the maximum length
#COMMENT 2P&1P: WTF!? Skal ikke disse alltid være like lange???????
n_y = n_max #- 1
y = np.random.randint(0, m, (b,n_y))
print(y)

#initialize a neural network based on the layers above
network = NeuralNetwork([embed_pos, feed_forward, attention, un_embed, softmax])
#and a loss function
loss = CrossEntropy()

#do a forward pass
Z = network.forward(X)

#compute the loss
print(Z.shape)
print(y.shape)
L = loss.forward(Z, y)

#get the derivative of the loss wrt Z
grad_Z = loss.backward()
print(grad_Z.shape)

#and perform a backward pass
_ = network.backward(grad_Z)

#and and do a gradient descent step
_ = network.step_gd(0.01)

[[5 7 4 6 2 6 2]
 [3 4 1 4 5 4 2]
 [4 5 7 2 4 5 4]
 [2 0 5 3 4 7 3]
 [1 5 2 0 5 0 7]
 [7 7 1 3 3 1 4]]
softmax input shape: (6, 7, 7)
softmax input shape: (6, 8, 7)
(6, 8, 7)
(6, 7)
7
onehotshape: (6, 8, 7)
(6, 8, 7)
z_l:(6, 8, 7)
grad: (6, 8, 7)
P: (6, 8, 7)
Q: (6, 1, 7)
z_l:(6, 7, 7)
grad: (6, 7, 7)
P: (6, 7, 7)
Q: (6, 1, 7)


ValueError: operands could not be broadcast together with remapped shapes [original->remapped]: (6,10,7)->(6,newaxis,newaxis) (7,7,6)->(7,newaxis,newaxis)  and requested shape (10,6)

In [ ]:
"""
Here you may add additional tests to for example:

- Check if the ['d'] keys in the parameter dictionaries are not None, or receive something when running backward pass
- Check if the parameters change when you perform a gradient descent step
- Check if the loss decreases when you perform a gradient descent step

This is voluntary, but could be useful.
"""

In [ ]:
#check if loss is non-negative
assert L >= 0, f"L={L}, expected L>=0"
assert grad_Z.shape == Z.shape, f"grad_Z.shape={grad_Z.shape}, expected {Z.shape}"

#check if onehot(y) gives zero loss
Y = onehot(y, m)
L = loss.forward(Y, y)
assert L < 1e-5, f"L={L}, expected L<1e-5"


In [ ]:
m = np.array([[[.29,.15,.10],
     [.21,.5,.04],
     [.15,.03,.11],
     [.13,.21,.43],
     [.22,.11,.32]],
     [
     [1,0,0],
     [0,1,0],
     [0,0,0],
     [0,0,0],
     [0,0,1]]])



loss = CrossEntropy()

loss.forward(m,np.array([[0,1,4],[0,1,4]]))
loss.backward()

array([[[-1.14942525, -0.        , -0.        ],
        [-0.        , -0.66666665, -0.        ],
        [-0.        , -0.        , -0.        ],
        [-0.        , -0.        , -0.        ],
        [-0.        , -0.        , -1.04166663]],

       [[-0.33333333, -0.        , -0.        ],
        [-0.        , -0.33333333, -0.        ],
        [-0.        , -0.        , -0.        ],
        [-0.        , -0.        , -0.        ],
        [-0.        , -0.        , -0.33333333]]])

In [ ]:
x_T = np.array([[[0,0,1],
      [0,0,0]]
     ,[[0,0,0],
       [1,0,0]]])

a = np.ones((3,2))
b = a.T

s = np.einsum('bad,ds,sq,bqk -> bak',x_T,a, b, np.transpose(x_T, axes=(0,2,1)), optimize=True)
for i in range(len(x_T)):
    print(s == (x_T[i]@a)@(b@x_T[i].T))

print(s)


[[[ True  True]
  [ True  True]]

 [[False  True]
  [ True False]]]
[[[False  True]
  [ True False]]

 [[ True  True]
  [ True  True]]]
[[[2. 0.]
  [0. 0.]]

 [[0. 0.]
  [0. 2.]]]
